# チャットボットの構築

[Build a Chatbot \| 🦜️🔗 LangChain](https://python.langchain.com/docs/tutorials/chatbot/)

## 概要
LLMを活用したチャットボットの設計と実装の例を紹介します。このチャットボットは会話を行い、[チャットモデル](https://python.langchain.com/docs/concepts/chat_models/)を使用して以前のやり取りを記憶することができます。

このチャットボットは会話を行うために言語モデルのみを使用することに注意してください。他にも関連する概念がいくつかあります:

- [Conversational RAG](https://python.langchain.com/docs/tutorials/qa_chat_history/): 外部データソースを使用してチャットボット体験を提供
- [エージェント](https://python.langchain.com/docs/tutorials/agents/): アクションを実行できるチャットボットを構築

このチュートリアルでは、これらの2つのより高度なトピックに役立つ基本をカバーしますが、必要に応じて直接そちらに進んでください。

In [0]:
%pip install -qU langchain[openai] langgraph mlflow
%restart_python

In [0]:
import mlflow

# MLflow Tracingの有効化
mlflow.langchain.autolog()

In [0]:
import os
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("demo-token-takaaki.yayoi", "openai_api_key")

## クイックスタート

In [0]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

まず、モデルを直接使用してみましょう。`ChatModel`はLangChainの「Runnable」のインスタンスであり、これらと対話するための標準インターフェースを提供します。モデルを単純に呼び出すには、メッセージのリストを`.invoke`メソッドに渡すことができます。

In [0]:
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel

model.invoke([HumanMessage(content="こんにちは！私はTakaです。")])

モデル自体には状態の概念がありません。例えば、フォローアップの質問をすると：

In [0]:
model.invoke([HumanMessage(content="私の名前はなんですか？")])

前の会話のターンを考慮せず、質問に答えることができないことがわかります。これはひどいチャットボット体験です！

これを回避するために、[会話の履歴](https://python.langchain.com/docs/concepts/chat_history/)全体をモデルに渡す必要があります。そうするとどうなるか見てみましょう：

In [0]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="こんにちは！私はTakaです。"),
        AIMessage(content="こんにちは、Takaさん！お会いできて嬉しいです。今日はどんなことをお話ししたいですか？"),
        HumanMessage(content="私の名前はなんですか？"),
    ]
)

**APIリファレンス**: [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html)

そして、良い応答が得られることがわかります！

これがチャットボットが会話的にやり取りする能力の基本的な考え方です。では、これを最適に実装するにはどうすればよいでしょうか？

## メッセージの永続化
[LangGraph](https://langchain-ai.github.io/langgraph/)は組み込みの永続化レイヤーを実装しており、複数の会話ターンをサポートするチャットアプリケーションに最適です。

チャットモデルを最小限のLangGraphアプリケーションでラップすることで、メッセージ履歴を自動的に永続化でき、マルチターンアプリケーションの開発が簡素化されます。

LangGraphにはシンプルなインメモリチェックポインタが付属しており、以下で使用します。詳細については[ドキュメント](https://langchain-ai.github.io/langgraph/concepts/persistence/)を参照してください。異なる永続化バックエンド（例：SQLiteやPostgres）の使用方法も含まれています。

In [0]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 新しいグラフを定義
workflow = StateGraph(state_schema=MessagesState)

# モデルを呼び出す関数を定義
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# グラフにノードを追加
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# メモリを追加
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [0]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

**APIリファレンス:** [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver) | [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)

次に、毎回runnableに渡す構成情報を作成する必要があります。この構成には、入力の一部ではないが依然として有用な情報が含まれています。この場合、`thread_id`を含めたいと考えています。これは次のようになります:

In [0]:
config = {"configurable": {"thread_id": "abc123"}}

これにより、単一のアプリケーションで複数の会話スレッドをサポートできるようになり、アプリケーションに複数のユーザーがいる場合によくある要件に対応できます。

次に、アプリケーションを呼び出すことができます:

In [0]:
query = "こんにちは！私はTakaです。"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # 出力には状態内のすべてのメッセージが含まれています

In [0]:
query = "私の名前はなんですか？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

素晴らしい！私たちのチャットボットは、私たちについての情報を覚えています。構成を変更して別の`thread_id`を参照するようにすると、新しい会話が始まることがわかります。

In [0]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

しかし、（データベースに保存しているため）元の会話に戻ることはいつでも可能です。

In [0]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

これが、複数のユーザーとの会話をサポートするチャットボットの方法です！

現在、私たちが行ったのは、モデルの周りに単純な永続化レイヤーを追加しただけです。プロンプトテンプレートを追加することで、チャットボットをより複雑でパーソナライズされたものにすることができます。

## プロンプトテンプレート

[プロンプトテンプレート](https://python.langchain.com/docs/concepts/prompt_templates/)は、生のユーザー情報をLLMが処理できる形式に変換するのに役立ちます。この場合、生のユーザー入力はメッセージだけであり、それをLLMに渡しています。これを少し複雑にしてみましょう。まず、カスタム指示を含むシステムメッセージを追加します（ただし、入力としてメッセージを受け取ります）。次に、メッセージ以外の入力も追加します。

システムメッセージを追加するために、`ChatPromptTemplate`を作成します。`MessagesPlaceholder`を利用して、すべてのメッセージを渡します。

In [0]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは海賊のように話します。できる限りの質問に答えてください。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

**APIリファレンス:** [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) | [MessagesPlaceholder](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

アプリケーションを更新してこのテンプレートを組み込みましょう：

In [0]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

同じ方法でアプリケーションを呼び出します：

In [0]:
config = {"configurable": {"thread_id": "abc345"}}
query = "はい！私はYayoiです。"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [0]:
query = "私の名前は？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

素晴らしい！プロンプトをもう少し複雑にしてみましょう。プロンプトテンプレートが次のようになったと仮定しましょう：

In [0]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは役に立つアシスタントです。{language}で可能な限りの力を尽くしてすべての質問に答えます。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

プロンプトに新しい入力`language`を追加したことに注意してください。アプリケーションには今や2つのパラメーター、入力`message`と`language`があります。この変更を反映するために、アプリケーションの状態を更新する必要があります：

In [0]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [0]:
config = {"configurable": {"thread_id": "abc456"}}
query = "はい、私はBobです。"
language = "英語"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

全体の状態が保持されるため、変更が必要ない場合は、`language`のようなパラメーターを省略できます：

In [0]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

## 会話履歴の管理

チャットボットを構築する際に理解すべき重要な概念の一つは、会話履歴の管理方法です。管理されないままにしておくと、メッセージのリストが無制限に増加し、LLMのコンテキストウィンドウをオーバーフローする可能性があります。したがって、渡すメッセージのサイズを制限するステップを追加することが重要です。

**重要なのは、メッセージ履歴から以前のメッセージを読み込んだ後、そしてプロンプトテンプレートの前にメッセージの制限処理を行うことです。**

これを行うために、プロンプトの前に`messages`のキーを適切に修正する簡単なステップを追加し、その新しいチェーンをメッセージ履歴クラスでラップします。

LangChainには、[メッセージのリストを管理](https://python.langchain.com/docs/how_to/#messages)するためのいくつかの組み込みヘルパーが付属しています。この場合、モデルに送信するメッセージの数を減らすために[trim\_messages](https://python.langchain.com/docs/how_to/trim_messages/)ヘルパーを使用します。トリマーでは、保持したいトークンの数や、システムメッセージを常に保持するかどうか、部分的なメッセージを許可するかどうかなどのパラメーターを指定できます：

In [0]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="あなたは良いアシスタントです"),
    HumanMessage(content="こんにちは！私はボブです"),
    AIMessage(content="こんにちは！"),
    HumanMessage(content="私はバニラアイスクリームが好きです"),
    AIMessage(content="いいですね"),
    HumanMessage(content="2 + 2 は何ですか"),
    AIMessage(content="4"),
    HumanMessage(content="ありがとう"),
    AIMessage(content="どういたしまして！"),
    HumanMessage(content="楽しんでいますか？"),
    AIMessage(content="はい！"),
]

trimmer.invoke(messages)

チェーンで使用するには、`messages`入力をプロンプトに渡す前にトリマーを実行するだけです。

In [0]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

今、モデルに名前を尋ねても、チャット履歴のその部分がトリミングされているため、モデルはそれを知りません：

In [0]:
config = {"configurable": {"thread_id": "abc567"}}
query = "私の名前は？"
language = "日本語"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

しかし、直近の数件のメッセージ内の情報について尋ねると、それを覚えています：

In [0]:
config = {"configurable": {"thread_id": "abc678"}}
query = "私が質問した感情の質問は何？"
language = "日本語"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

## ストリーミング

これで動作するチャットボットを手に入れました。ただし、チャットボットアプリケーションの非常に重要なUXの考慮事項の1つは、ストリーミングです。LLMは応答するのに時間がかかることがありますので、ユーザーエクスペリエンスを向上させるために、ほとんどのアプリケーションが行うことの1つは、生成されるトークンごとにストリームバックすることです。これにより、ユーザーは進捗状況を確認できます。

実際、これを行うのは非常に簡単です！

デフォルトでは、LangGraphアプリケーションの`.stream`はアプリケーションステップをストリーム化します-- この場合、モデル応答の単一ステップです。`stream_mode="messages"`を設定すると、出力トークンをストリーム化できます：

In [0]:
config = {"configurable": {"thread_id": "abc789"}}
query = "はい、私はToddです。ジョークを教えてください。"
language = "日本語"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # モデルのレスポンスのみにフィルタリング
        print(chunk.content, end="|")